In [26]:
import ee
ee.Initialize()

In [27]:
# Define the time period for the NDVI composite
year = 2022

# Set start and end dates using the year variable
start_date = f'{year}-01-01'
end_date = f'{year}-12-31'

# Load the MODIS NDVI dataset and filter by date
modis_ndvi = ee.ImageCollection('MODIS/006/MOD13A2') \
              .filterDate(start_date, end_date) \
              .select('NDVI')

# Calculate the mean NDVI over the specified period
ndvi_composite = modis_ndvi.mean()

# Scale NDVI values (MODIS NDVI is scaled by a factor of 0.0001)
ndvi_composite = ndvi_composite.multiply(0.0001).rename('NDVI')

In [ ]:
# Define the export parameters
task = ee.batch.Export.image.toDrive(
    image=ndvi_composite,
    description=f'Global_Annual_NDVI_{year}',  # Use f-string to insert year
    scale=1000,                               # Set the scale to 1000m (1km resolution)
    region=ee.Geometry.BBox(-180, -90, 180, 90),  # Global extent
    crs='EPSG:4326',                          # WGS84
    fileFormat='GeoTIFF',
    maxPixels=1e13                             # Increase this to handle global data
)

# Start the export task
task.start()

In [7]:
import ee
ee.Initialize()

# Load the Copernicus GLO-30 DEM dataset
copernicus_dem = ee.ImageCollection('COPERNICUS/DEM/GLO30').select('DEM').mosaic()

# Define the region to include Greenland and northern areas
region = ee.Geometry.BBox(-180, -90, 180, 90)

# Reproject and resample the DEM to 1 km resolution using mean reducer
coarse_copernicus_dem = copernicus_dem.reproject(
    crs='EPSG:4326',  # WGS84 projection
    scale=1000  # Set the new resolution to 1 km (1000 meters)
).reduceResolution(
    reducer=ee.Reducer.mean(),  # Compute the mean for 1 km pixels
    maxPixels=1024  # Control pixel aggregation
)

# Export the resampled image at 1 km resolution with mean values
task = ee.batch.Export.image.toDrive(
    image=coarse_copernicus_dem,
    description='Global_Elevation_Copernicus_1km_avg_northern',  # Description for the export task
    scale=1000,  # Use the 1 km resolution for export
    region=region,  # Adjusted region to include Greenland and northern latitudes
    crs='EPSG:4326',  # WGS84 projection
    fileFormat='GeoTIFF',
    maxPixels=1e13  # Allow large global data exports
)

# Start the export task
task.start()
print("Export started. Check Google Earth Engine Tasks tab to monitor progress.")


Export started. Check Google Earth Engine Tasks tab to monitor progress.
